<a href="https://colab.research.google.com/github/niit-ibm/lt4-pe-lab1/blob/main/sim1_handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 1: Prompt Engineering for Project Status Updates
## Hands-On Exercise Using IBM Granite Model via Replicate

### Scenario
You are a project intern at a mid-sized IT services company, supporting several project teams. You draft 120-word status updates for internal and external stakeholders weekly. Project leads send information in different formats, making AI-generated drafts inconsistent.


## Setup: Install Required Libraries

First, install the necessary libraries as used in the reference implementation.

In [ ]:
# Install required libraries
!pip install "langchain_community<0.3.0" replicate ipywidgets ibm-granite-community --quiet

## Configure Environment and Import Libraries

In [1]:
import os
import replicate
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

## Set Replicate API Token

You need a Replicate API token to use the IBM Granite model. Get your token from [replicate.com](https://replicate.com/account/api-tokens)

In [ ]:
# Use the utility function to get from environment or prompt
replicate_api_token = get_env_var("REPLICATE_API_TOKEN")
os.environ["REPLICATE_API_TOKEN"] = replicate_api_token

---
## Task 1: Initialize the IBM Granite Foundation Model

We'll use the **IBM Granite 3.3 8B Instruct** model via Replicate, matching the reference implementation.

In [ ]:
# Model configuration - exactly as in reference
MODEL_NAME = "ibm-granite/granite-3.3-8b-instruct"
MAX_TOKENS = 1024
TEMPERATURE = 0.2

# Initialize the model
llm = Replicate(
    model=MODEL_NAME,
    model_kwargs={
        "max_tokens": MAX_TOKENS,
        "temperature": TEMPERATURE
    }
)

print(f"✅ Model initialized: {MODEL_NAME}")
print(f"   Max Tokens: {MAX_TOKENS}")
print(f"   Temperature: {TEMPERATURE}")

## Sample Project Input Data

Here are two different formats of project updates you might receive from project leads:

In [ ]:
# Input A: Bulleted, messy, partial notes
input_a = """Project: Alpha CRM Upgrade — Weekly Update Notes
• Login issue fixed for 80% users, pending bug #3421
• Deployment for Phase 2 pushed from 15th to 19th
• Client appreciated responsiveness
• Risk: insufficient test data for UAT
• Next steps: Dev team to finalize API changes, Testing team to complete regression
• Need to highlight dependency on Data Engineering
"""

# Input B: Narrative-style email
input_b = """Forwarded message from Priya (Project Lead — Employee Portal Revamp)
Here's the update you asked for — We completed the UI redesign for the feedback module and shared it with the client yesterday. They're happy with the look and want us to proceed. We're still waiting for HR to confirm the final list of features for Phase 2, so timelines may shift a bit. Testing is ongoing. No major blockers right now, but remind the team that the API documentation still needs review. Let's capture that as a key reminder.
"""

print("Sample inputs loaded successfully!")
print("\nYou can use 'input_a' or 'input_b' in the exercises below.")

---
## Task 2: Start with a Direct Prompt

### Teaching Moment: Direct Prompts

A ** Direct Prompt** provides minimal guidance to the model. Let's see what happens when we use a simple, direct prompt.

In [ ]:
# Direct, zero-shot prompt
zero_shot_prompt = "Write a 120-word weekly project update."

print("🔵 TASK 2: Zero-Shot Prompting")
print("="*60)
print(f"Prompt: {zero_shot_prompt}\n")

# Get response from model
zero_shot_response = llm.invoke(zero_shot_prompt)

print("Response:")
print("-"*60)
print(zero_shot_response)
print("-"*60)

### Task 2: Analysis

**Observe the limitations:**
- Output is vague and generic
- No specific structure
- May repeat wording from the input
- Lacks context about the project

**Conclusion:** A direct, zero-shot prompt is insufficient for generating structured, reliable project updates. We need to add more guidance.

---
## Task 3: Refine to a Task-Specific Prompt with Structured Instructions

Now let's add:
- Clear task description
- Context about the audience
- Desired output format

We'll use one of the sample inputs to make it more realistic.

In [ ]:
# Task-specific prompt with structure
def create_structured_prompt(project_input):
    prompt = f"""Create a 120-word weekly project update for internal stakeholders.
Include: current status, completed tasks, risks, and next steps.

Based on this input:
{project_input}
"""
    return prompt

# Let's use Input A for this task
structured_prompt = create_structured_prompt(input_a)

print("🟢 TASK 3: Task-Specific Prompt with Structured Instructions")
print("="*60)
print("Prompt:")
print(structured_prompt)
print("\nGenerating response...\n")

# Get response from model
structured_response = llm.invoke(structured_prompt)

print("Response:")
print("-"*60)
print(structured_response)
print("-"*60)

### Task 3: Analysis

**Improvements observed:**
- Output becomes clearer and more focused
- More relevant information appears
- Better structure emerges

**Remaining issues:**
- May not match the professional tone expected
- Format could be more consistent

**Conclusion:** Task-specific prompts with structured instructions improve output quality, but we need to refine further by adding role, tone, and format specifications.

---
## Task 4: Refine Using Advanced Prompt Engineering Techniques

Now we'll apply best practices:
- **Role prompting**: Define who the AI is
- **Tone specification**: Set the communication style
- **Format constraints**: Specify exact output format

This creates a comprehensive, well-engineered prompt.

In [ ]:
# Advanced prompt with role, tone, and format
def create_advanced_prompt(project_input):
    prompt = f"""You are a project reporting assistant for an IT services company.

Task: Generate a 120-word weekly project update for internal stakeholders.

Tone: Concise and professional.

Format: Use bullet points for the following sections:
- Current Status
- Completed Tasks
- Risks/Issues
- Next Steps

Input information:
{project_input}

Generate the update now:
"""
    return prompt

# Test with Input A
advanced_prompt = create_advanced_prompt(input_a)

print("🟣 TASK 4: Advanced Prompt Engineering")
print("="*60)
print("Prompt:")
print(advanced_prompt)
print("\nGenerating response...\n")

# Get response from model
advanced_response = llm.invoke(advanced_prompt)

print("Response:")
print("-"*60)
print(advanced_response)
print("-"*60)

### Task 4: Analysis

**Results:**
- Highly structured output
- Professional tone that matches workplace expectations
- No irrelevant details or repetition
- Consistent format across different inputs

**Conclusion:** By combining task description, context, output format, role, and tone, you produce polished, stakeholder-ready project updates. This demonstrates how layering multiple prompting techniques leads to high-quality outputs.

---
## Comparison: All Three Approaches Side by Side

In [8]:
# Create a comparison display
def display_comparison():
    html_content = f"""
    <style>
        .comparison-container {{
            display: flex;
            flex-direction: column;
            gap: 20px;
            font-family: Arial, sans-serif;
        }}
        .comparison-box {{
            border: 2px solid #ddd;
            border-radius: 8px;
            padding: 15px;
            background-color: #f9f9f9;
        }}
        .comparison-title {{
            font-weight: bold;
            font-size: 16px;
            margin-bottom: 10px;
            color: #333;
        }}
        .zero-shot {{ border-left: 5px solid #FF6B6B; }}
        .structured {{ border-left: 5px solid #4ECDC4; }}
        .advanced {{ border-left: 5px solid #95E1D3; }}
    </style>
    <div class="comparison-container">
        <div class="comparison-box zero-shot">
            <div class="comparison-title">Task 2: Zero-Shot Prompt</div>
            <div><strong>Limitations:</strong> Vague, no structure, generic content</div>
        </div>
        <div class="comparison-box structured">
            <div class="comparison-title">Task 3: Structured Prompt</div>
            <div><strong>Improvements:</strong> Clear structure, relevant information, better organization</div>
        </div>
        <div class="comparison-box advanced">
            <div class="comparison-title">Task 4: Advanced Prompt Engineering</div>
            <div><strong>Best Results:</strong> Professional tone, perfect structure, stakeholder-ready</div>
        </div>
    </div>
    """
    display(HTML(html_content))

display_comparison()

---
## Practice Exercise: Try with Input B

Now it's your turn! Apply what you've learned with the narrative-style email (Input B).

In [9]:
# Exercise: Generate update using advanced prompt with Input B
practice_prompt = create_advanced_prompt(input_b)

print("📝 PRACTICE EXERCISE: Employee Portal Revamp Project")
print("="*60)
print("\nGenerating update from narrative-style email...\n")

practice_response = llm.invoke(practice_prompt)

print("Generated Update:")
print("-"*60)
print(practice_response)
print("-"*60)

📝 PRACTICE EXERCISE: Employee Portal Revamp Project

Generating update from narrative-style email...

Generated Update:
------------------------------------------------------------
▸ **Current Status**
- UI redesign for the feedback module completed and approved by the client.
- Ongoing testing phase.
- Waiting for HR to confirm final features for Phase 2.

▸ **Completed Tasks**
- Finished UI redesign for the feedback module.
- Shared the redesigned module with the client, receiving positive feedback.

▸ **Risks/Issues**
- Potential timeline adjustments due to pending HR feature confirmation for Phase 2.
- API documentation requires review, though not currently blocking progress.

▸ **Next Steps**
- Finalize API documentation review.
- Continue with the testing phase.
- Await HR confirmation on Phase 2 features to solidify timelines.
- Keep the team informed of any updates regarding feature confirmation and documentation review.
---------------------------------------------------------

---
## Interactive Refinement Function

Based on the reference implementation, we can also refine outputs iteratively.

In [10]:
def refine_output(original_output, refinement_request):
    """
    Refine previously generated output based on user request.
    Similar to refine_output_with_fewshot from reference.
    """
    refinement_prompt = f"""You are a project reporting assistant.

Here is a project update that was previously generated:
{original_output}

Please refine this update based on the following request:
{refinement_request}

Provide the refined version:
"""

    refined_response = llm.invoke(refinement_prompt)
    return refined_response

print("✅ Refinement function ready!")
print("\nExample usage:")
print("refined = refine_output(advanced_response, 'Make it more concise and add emoji bullets')")

✅ Refinement function ready!

Example usage:
refined = refine_output(advanced_response, 'Make it more concise and add emoji bullets')


## Try Refinement

In [ ]:
# Example refinement
refinement_request = "Make the update more concise and use a numbered list instead of bullet points"

print("🔄 REFINEMENT EXAMPLE")
print("="*60)
print(f"Refinement Request: {refinement_request}\n")

refined_output = refine_output(advanced_response, refinement_request)

print("Refined Output:")
print("-"*60)
print(refined_output)
print("-"*60)

---
## Summary and Key Takeaways

### What You Learned:

1. **Zero-Shot Prompting**: Simple direct prompts produce vague, unstructured outputs

2. **Structured Prompting**: Adding task description, context, and format improves clarity

3. **Advanced Prompt Engineering**: Combining role, tone, and detailed format specifications produces professional, stakeholder-ready outputs

4. **Iterative Refinement**: You can further refine outputs based on specific requirements

### Best Practices:
- Define the AI's role clearly
- Specify desired tone and format
- Provide context and examples
- Structure your requirements explicitly
- Iterate and refine as needed

### Technical Stack Used:
- **Model**: IBM Granite 3.3 8B Instruct
- **Platform**: Replicate
- **Libraries**: langchain_community, replicate, ibm-granite-community
- **Parameters**: Temperature=0.2, Max Tokens=1024

---
## Additional Exercises

Try these on your own:

1. **Create your own project input** in a different format and generate an update

2. **Experiment with different tones**: formal, casual, executive-level

3. **Try different output formats**: paragraphs, tables, executive summaries

4. **Compare temperature settings**: Try 0.0, 0.5, and 0.8 to see how creativity changes

5. **Multi-project updates**: Combine multiple project inputs into a single consolidated update

In [13]:
# Space for your experiments
# TODO: Add your own project input and prompts here

my_project_input = """
# Add your project information here
"""

# Your custom prompt
my_prompt = """
# Create your prompt here
"""

# Uncomment to run:
# my_response = llm.invoke(my_prompt)
# print(my_response)